# Imports

In [1]:
import pandas as pd
import numpy as np

from autogluon.tabular import TabularPredictor

import os
import gc
train_path = r'D:/KaggleTabular/August/train.csv'
test_path = r'D:/KaggleTabular/August/test.csv'
submission_path = r'D:/KaggleTabular/August/sample_submission.csv'
RS = 69420
os.environ['NUMEXPR_MAX_THREADS'] = '16'

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
train = pd.read_csv(train_path, index_col=0)
train.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
id,,,,,,,,,,,,,,,,,,,,,
0,-0.002350,59,0.766739,-1.350460,42.2727,16.68570,30.3599,1.267300,0.392007,1.09101,...,-42.43990,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.29658,10.48980,15
1,0.784462,145,-0.463845,-0.530421,27324.9000,3.47545,160.4980,0.828007,3.735860,1.28138,...,-184.13200,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.36470,1.14383,3
2,0.317816,19,-0.432571,-0.382644,1383.2600,19.71290,31.1026,-0.515354,34.430800,1.24210,...,7.43721,37.218100,3.25339,0.337934,0.615037,2.216760,0.745268,1.69679,12.30550,6
3,0.210753,17,-0.616454,0.946362,-119.2530,4.08235,185.2570,1.383310,-47.521400,1.09130,...,9.66778,0.626942,1.49425,0.517513,-10.222100,2.627310,0.617270,1.45645,10.02880,2
4,0.439671,20,0.968126,-0.092546,74.3020,12.30650,72.1860,-0.233964,24.399100,1.10151,...,290.65700,15.604300,1.73557,-0.476668,1.390190,2.195740,0.826987,1.78485,7.07197,1


In [3]:
gc.collect()

17

In [4]:
label = 'loss'
eval_metric='rmse'
save_path = r'./AutoGlonAugust/'
timeLimit = 3600 * 12
search_strategy = 'auto'
num_trials = 1000

In [5]:
hyperparameter_kwargs = {
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy
}

In [6]:
models={
    #'NN': {},
    'GBM': [{'device':'gpu', 'extra_trees': False, 'ag_args': {'name_suffix': 'XT'}},{'device':'gpu'},'GBMLarge',],
    'CAT': {'task_type':'GPU'},
    'XGB': {'tree_method':'gpu_hist'},
    #'FASTAI': {}
}

In [7]:
predictor = TabularPredictor(
    label=label,
    problem_type='regression',
    eval_metric=eval_metric,
    path=save_path,
    verbosity=2
)

In [8]:
%%time
predictor.fit(
    train,
    hyperparameters=models,
    presets='best_quality',
    keep_only_best=True,
    refit_full=True,
    save_space=True,
    hyperparameter_tune_kwargs=hyperparameter_kwargs,
    time_limit=timeLimit
)

Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 43200s
AutoGluon will save models to "./AutoGlonAugust/\"
AutoGluon Version:  0.2.0
Train Data Rows:    250000
Train Data Columns: 100
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11836.64 MB
	Train Data (Original)  Memory Usage: 200.0 MB (1.7% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 95 | ['f0', 'f2', 'f3', 'f4', 'f5', ...]
		('int', [])   :  5 | ['f1', 'f16', 'f27', 'f5

[1000]	train_set's rmse: 7.50879	valid_set's rmse: 7.92215
[1000]	train_set's rmse: 7.62717	valid_set's rmse: 7.93238
[2000]	train_set's rmse: 7.40754	valid_set's rmse: 7.91949
[1000]	train_set's rmse: 7.36689	valid_set's rmse: 7.91964
[2000]	train_set's rmse: 6.98042	valid_set's rmse: 7.91594
[1000]	train_set's rmse: 7.59126	valid_set's rmse: 7.92157
[2000]	train_set's rmse: 7.36044	valid_set's rmse: 7.91638
[3000]	train_set's rmse: 7.14923	valid_set's rmse: 7.91496
[1000]	train_set's rmse: 7.56369	valid_set's rmse: 7.92636
[2000]	train_set's rmse: 7.30715	valid_set's rmse: 7.91743
[3000]	train_set's rmse: 7.08239	valid_set's rmse: 7.9151


	Ran out of time, early stopping on iteration 3815. Best iteration is:
	[3541]	train_set's rmse: 6.96666	valid_set's rmse: 7.91454
	Time limit exceeded
Fitted model: LightGBMXT_BAG_L1\T0 ...
	-7.9257	 = Validation root_mean_squared_error score
	7.03s	 = Training runtime
	0.09s	 = Validation runtime
Fitted model: LightGBMXT_BAG_L1\T1 ...
	-7.9177	 = Validation root_mean_squared_error score
	24.8s	 = Training runtime
	0.25s	 = Validation runtime
Fitted model: LightGBMXT_BAG_L1\T2 ...
	-7.9315	 = Validation root_mean_squared_error score
	8.38s	 = Training runtime
	0.07s	 = Validation runtime
Fitted model: LightGBMXT_BAG_L1\T3 ...
	-7.9287	 = Validation root_mean_squared_error score
	4.66s	 = Training runtime
	0.05s	 = Validation runtime
Fitted model: LightGBMXT_BAG_L1\T4 ...
	-7.9257	 = Validation root_mean_squared_error score
	16.24s	 = Training runtime
	0.11s	 = Validation runtime
Fitted model: LightGBMXT_BAG_L1\T5 ...
	-7.92	 = Validation root_mean_squared_error score
	14.91s	 = Traini

[1000]	train_set's rmse: 7.53988	valid_set's rmse: 7.92943
[2000]	train_set's rmse: 7.25928	valid_set's rmse: 7.92081
[3000]	train_set's rmse: 7.01872	valid_set's rmse: 7.91916
[4000]	train_set's rmse: 6.79554	valid_set's rmse: 7.91822
[5000]	train_set's rmse: 6.5881	valid_set's rmse: 7.9181
[1000]	train_set's rmse: 7.09799	valid_set's rmse: 7.9181
[1000]	train_set's rmse: 7.7742	valid_set's rmse: 7.93254
[2000]	train_set's rmse: 7.68355	valid_set's rmse: 7.92167
[3000]	train_set's rmse: 7.60326	valid_set's rmse: 7.91872
[4000]	train_set's rmse: 7.52697	valid_set's rmse: 7.91666
[5000]	train_set's rmse: 7.45288	valid_set's rmse: 7.91581
[6000]	train_set's rmse: 7.38121	valid_set's rmse: 7.91534
[7000]	train_set's rmse: 7.31126	valid_set's rmse: 7.91542
[1000]	train_set's rmse: 7.39763	valid_set's rmse: 7.92265
[2000]	train_set's rmse: 7.03217	valid_set's rmse: 7.92159
[3000]	train_set's rmse: 6.7043	valid_set's rmse: 7.92078


	Time limit exceeded
Fitted model: LightGBM_BAG_L1\T0 ...
	-7.9257	 = Validation root_mean_squared_error score
	6.78s	 = Training runtime
	0.09s	 = Validation runtime
Fitted model: LightGBM_BAG_L1\T1 ...
	-7.9176	 = Validation root_mean_squared_error score
	173.59s	 = Training runtime
	2.65s	 = Validation runtime
Fitted model: LightGBM_BAG_L1\T2 ...
	-7.9335	 = Validation root_mean_squared_error score
	11.56s	 = Training runtime
	0.07s	 = Validation runtime
Fitted model: LightGBM_BAG_L1\T3 ...
	-7.9179	 = Validation root_mean_squared_error score
	41.71s	 = Training runtime
	0.27s	 = Validation runtime
Fitted model: LightGBM_BAG_L1\T4 ...
	-7.9545	 = Validation root_mean_squared_error score
	5.42s	 = Training runtime
	0.04s	 = Validation runtime
Fitted model: LightGBM_BAG_L1\T5 ...
	-7.915	 = Validation root_mean_squared_error score
	68.37s	 = Training runtime
	2.3s	 = Validation runtime
Fitted model: LightGBM_BAG_L1\T6 ...
	-7.9198	 = Validation root_mean_squared_error score
	92.64s	 =

...

[1000]	train_set's rmse: 7.72314	valid_set's rmse: 7.85178


	-7.8371	 = Validation root_mean_squared_error score
	191.23s	 = Training runtime
	1.48s	 = Validation runtime
Fitting model: LightGBM_BAG_L2\T28 ... Training model for up to 18222.62s of the 18222.36s of remaining time.
	-7.8377	 = Validation root_mean_squared_error score
	38.59s	 = Training runtime
	0.57s	 = Validation runtime
Fitting model: LightGBM_BAG_L2\T29 ... Training model for up to 18186.21s of the 18185.95s of remaining time.
	-7.837	 = Validation root_mean_squared_error score
	116.56s	 = Training runtime
	1.06s	 = Validation runtime
Fitting model: LightGBM_BAG_L2\T30 ... Training model for up to 18078.38s of the 18078.12s of remaining time.
	-7.8372	 = Validation root_mean_squared_error score
	138.94s	 = Training runtime
	1.03s	 = Validation runtime
Fitting model: LightGBM_BAG_L2\T31 ... Training model for up to 17952.14s of the 17951.88s of remaining time.
	-7.8385	 = Validation root_mean_squared_error score
	30.39s	 = Training runtime
	0.52s	 = Validation runtime
Fitting 

[1000]	train_set's rmse: 7.7424	valid_set's rmse: 7.8533


	-7.8376	 = Validation root_mean_squared_error score
	163.87s	 = Training runtime
	1.4s	 = Validation runtime
Fitting model: LightGBM_BAG_L2\T39 ... Training model for up to 17522.63s of the 17522.37s of remaining time.
	-7.8374	 = Validation root_mean_squared_error score
	111.05s	 = Training runtime
	0.89s	 = Validation runtime
Fitting model: LightGBM_BAG_L2\T40 ... Training model for up to 17420.91s of the 17420.64s of remaining time.
	-7.8377	 = Validation root_mean_squared_error score
	76.28s	 = Training runtime
	0.73s	 = Validation runtime
Fitting model: CatBoost_BAG_L2\T0 ... Training model for up to 17351.67s of the 17351.41s of remaining time.
	-7.841	 = Validation root_mean_squared_error score
	35.48s	 = Training runtime
	0.11s	 = Validation runtime
Fitting model: CatBoost_BAG_L2\T1 ... Training model for up to 17320.81s of the 17320.55s of remaining time.
	-7.841	 = Validation root_mean_squared_error score
	64.24s	 = Training runtime
	0.12s	 = Validation runtime
Fitting model

Wall time: 9h 4min 8s


In [9]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                     model  score_val  pred_time_val      fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      WeightedEnsemble_L3  -7.836073     255.406494  19449.809974                0.010070          15.292959            3       True        129
1    LightGBMXT_BAG_L2\T16  -7.836799     202.957029  17665.430644                1.174989         152.065798            2       True        121
2      LightGBM_BAG_L2\T11  -7.837035     202.628789  17599.761035                0.846750          86.396189            2       True        123
3    LightGBMXT_BAG_L2\T18  -7.837048     203.348988  17773.527536                1.566949         260.162690            2       True        122
4      LightGBM_BAG_L2\T19  -7.837082     202.444100  17561.067316                0.662060          47.702470            2       True        124
5        XGBoost_BAG_L2\T2  -7.837316     218.621715  18188.356116  

# Predict Model

In [10]:
submission = pd.read_csv(submission_path, index_col=0)
submission.head()

C:\Users\Elite_PC\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,loss
id,
250000,0
250001,0
250002,0
250003,0
250004,0


In [11]:
test = pd.read_csv(test_path, index_col=0)
test.head()

C:\Users\Elite_PC\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
id,,,,,,,,,,,,,,,,,,,,,
250000,0.812665,15,-1.239120,-0.893251,295.5770,15.87120,23.04360,0.942256,29.898000,1.11394,...,0.446389,-422.332,-1.44630,1.69075,1.059300,-3.010570,1.94664,0.529470,1.386950,8.78767
250001,0.190344,131,-0.501361,0.801921,64.8866,3.09703,344.80500,0.807194,38.421900,1.09695,...,0.377179,10352.200,21.06270,1.84351,0.251895,4.440570,1.90309,0.248534,0.863881,11.79390
250002,0.919671,19,-0.057382,0.901419,11961.2000,16.39650,273.24000,-0.003300,37.940000,1.15222,...,0.990140,3224.020,-2.25287,1.55100,-0.559157,17.838600,1.83385,0.931796,2.336870,9.05400
250003,0.860985,19,-0.549509,0.471799,7501.6000,2.80698,71.08170,0.792136,0.395235,1.20157,...,1.396880,9689.760,14.77150,1.41390,0.329272,0.802437,2.23251,0.893348,1.359470,4.84833
250004,0.313229,89,0.588509,0.167705,2931.2600,4.34986,1.57187,1.118300,7.754630,1.16807,...,0.862502,2693.350,44.18050,1.58020,-0.191021,26.253000,2.68238,0.361923,1.532800,3.70660


In [ ]:
y_preds = predictor.predict_proba(test)

C:\Users\Elite_PC\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
submission.loss = y_preds
submission.head()

In [ ]:
submission.to_csv('submission.csv')

In [ ]:
# This code will submit the model to the competition if you have installed kaggle
!kaggle competitions submit -c tabular-playground-series-aug-2021 -f submission.csv -m "AutoGluon Model"

In [ ]:
import time
from IPython.core.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))
time.sleep(10)

import os
os.system("shutdown /s /t 1")